3.2. Textual Analysis

In [ ]:
# Gerekli kütüphaneleri yükleme
import pandas as pd
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Veri setini yükleme
data = pd.read_csv('veri_seti.csv')

# Metin önişleme
def preprocess_text(text):
    # Metni küçük harflere çevirme
    text = text.lower()
    # Özel karakterleri ve noktalama işaretlerini kaldırma
    text = text.replace("'", "")
    text = text.replace(".", "")
    text = text.replace(",", "")
    return text

# Metin verilerini önişleme
data['preprocessed_text'] = data['text'].apply(preprocess_text)

# Duygu analizi yapma
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        return 'positive'
    elif sentiment < 0:
        return 'negative'
    else:
        return 'neutral'

# Metin verilerini duygu analizine tabi tutma
data['sentiment'] = data['preprocessed_text'].apply(analyze_sentiment)

# Metin verilerini özelliklere dönüştürme
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(data['preprocessed_text'])

# Eğitim ve test verilerini ayırma
X_train, X_test, y_train, y_test = train_test_split(features, data['sentiment'], test_size=0.2, random_state=42)

# Sınıflandırma modelini eğitme
model = LogisticRegression()
model.fit(X_train, y_train)

# Test verilerini kullanarak tahmin yapma
y_pred = model.predict(X_test)

# Sınıflandırma raporunu gösterme
print(classification_report(y_test, y_pred))


3.3. Visual and Video Analysis

In [ ]:
# Gerekli kütüphaneleri yükleme
import pandas as pd
import cv2
import numpy as np
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Model

# Veri setini yükleme
data = pd.read_csv('veri_seti.csv')

# Resim verilerini yükleme ve önişleme
def load_and_preprocess_image(path):
    img = image.load_img(path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

# Resim verilerini özelliklere dönüştürme
def extract_image_features(img):
    model = VGG16(weights='imagenet', include_top=False)
    model = Model(inputs=model.input, outputs=model.get_layer('block4_pool').output)
    features = model.predict(img)
    features = features.flatten()
    return features

# Veri setindeki resimlerin özelliklerini çıkarma
image_features = []
for path in data['image_paths']:
    img = load_and_preprocess_image(path)
    features = extract_image_features(img)
    image_features.append(features)

# Video verilerini yükleme ve önişleme
def load_and_preprocess_video(path):
    cap = cv2.VideoCapture(path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (224, 224))
            frame = image.img_to_array(frame)
            frame = np.expand_dims(frame, axis=0)
            frame = preprocess_input(frame)
            frames.append(frame)
        else:
            break
    cap.release()
    video = np.concatenate(frames, axis=0)
    return video

# Video verilerini özelliklere dönüştürme
def extract_video_features(video):
    model = VGG16(weights='imagenet', include_top=False)
    model = Model(inputs=model.input, outputs=model.get_layer('block4_pool').output)
    features = model.predict(video)
    features = np.mean(features, axis=0)
    features = features.flatten()
    return features

# Veri setindeki videoların özelliklerini çıkarma
video_features = []
for path in data['video_paths']:
    video = load_and_preprocess_video(path)
    features = extract_video_features(video)
    video_features.append(features)

# Özellikleri birleştirme
features = np.concatenate((image_features, video_features), axis=1)

# Eğitim ve test verilerini ayırma
X_train, X_test, y_train, y_test = train_test_split(features, data['emotion'], test_size=0.2, random_state=42)

# Sınıflandırma modelini eğitme
model = LogisticRegression()
model.fit(X_train, y_train)

# Test verilerini kullanarak tahmin yapma
y_pred = model.predict(X_test)

# Sınıflandırma raporunu gösterme
print(classification_report(y_test, y_pred))


3.4. Audio Analysis

In [ ]:
# Gerekli kütüphaneleri yükleme
import pandas as pd
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Veri setini yükleme
data = pd.read_csv('veri_seti.csv')

# Ses dosyasını yükleme ve özellik çıkarma
def extract_audio_features(file_path):
    # Ses dosyasını yükleme
    audio, sr = librosa.load(file_path, duration=3)  # 3 saniyelik bir süreyle yükleme yapabilirsiniz
    
    # Ses özelliklerini çıkarma
    features = []
    
    # MFCC özelliklerini çıkarmak için:
    mfcc = librosa.feature.mfcc(audio, sr=sr)
    features.append(np.mean(mfcc))
    features.append(np.std(mfcc))
    features.append(np.min(mfcc))
    features.append(np.max(mfcc))
    
    
    return features

# Ses dosyalarının özelliklerini çıkarma
audio_features = []
for path in data['audio_paths']:
    features = extract_audio_features(path)
    audio_features.append(features)

# Özellikleri birleştirme
features = np.array(audio_features)

# Eğitim ve test verilerini ayırma
X_train, X_test, y_train, y_test = train_test_split(features, data['emotion'], test_size=0.2, random_state=42)

# Sınıflandırma modelini eğitme
model = LogisticRegression()
model.fit(X_train, y_train)

# Test verilerini kullanarak tahmin yapma
y_pred = model.predict(X_test)

# Sınıflandırma raporunu gösterme
print(classification_report(y_test, y_pred))
